# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import keras
import tensorflow
print('tensorflow version: {}'.format(tensorflow.__version__))
print('keras version: {}'.format(keras.__version__))

Using TensorFlow backend.


tensorflow version: 1.15.0
keras version: 2.0.9


In [3]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7800133315947393566
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7313188906160029653
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5733359020052250468
physical_device_desc: "device: XLA_GPU device"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [5]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    
    tokenizer = Tokenizer(filters = '')
    tokenizer.fit_on_texts(x)
    tokenized = tokenizer.texts_to_sequences(x)
    
    return tokenized, tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'.': 1, 'the': 2, 'quick': 3, 'a': 4, 'brown': 5, 'fox': 6, 'jumps': 7, 'over': 8, 'lazy': 9, 'dog': 10, 'by': 11, 'jove': 12, ',': 13, 'my': 14, 'study': 15, 'of': 16, 'lexicography': 17, 'won': 18, 'prize': 19, 'this': 20, 'is': 21, 'short': 22, 'sentence': 23}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [2, 3, 5, 6, 7, 8, 2, 9, 10, 1]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [11, 12, 13, 14, 3, 15, 16, 17, 18, 4, 19, 1]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [20, 21, 4, 22, 23, 1]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    padded_x = pad_sequences(x, maxlen = length, padding = 'post', truncating = 'post' )
    
    return padded_x
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 2  3  5  6  7  8  2  9 10  1]
  Output: [ 2  3  5  6  7  8  2  9 10  1  0  0]
Sequence 2 in x
  Input:  [11 12 13 14  3 15 16 17 18  4 19  1]
  Output: [11 12 13 14  3 15 16 17 18  4 19  1]
Sequence 3 in x
  Input:  [20 21  4 22 23  1]
  Output: [20 21  4 22 23  1  0  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)+1 # add 1 due to padding
french_vocab_size = len(french_tokenizer.word_index)+1 # add 1 due to padding

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 17
Max French sentence length: 23
English vocabulary size: 228
French vocabulary size: 355


Note that the french_words_counter that was created at the begining of this exercise has 355 keys but french_tokenizer only has 354 keys. Apparently, there is some word missing from the french_tokenizer. Let's find out which is this word:

In [11]:
not_found = []
for word in french_words_counter.keys():
    if not word in french_tokenizer.word_index.keys():
        print('Not found in french_tokenizer: {}'.format(word))
        not_found.append(word)

Not found in french_tokenizer: États-unis


This is perhaps because somehow 'États-unis' starts with an upper-case letter. I guess this is appropriately converted to lower case by the tokenizer and therefore identified with 'états-unis' by the tokenizer. Let's check if this is true.

In [12]:
# convert the word in not_found using the french_tokenizer
french_tokenizer.texts_to_sequences(not_found)

[[33]]

We see that the french_tokenizer assigns an index 33 to the word. Let's see if this is same as the index for 'états-unis'.

In [13]:
french_tokenizer.texts_to_sequences(['états-unis'])

[[33]]

Voila! We see that indeed  'États-unis' is identified with 'états-unis' by the tokenizer.

## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [14]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [15]:
def indices_to_text(idx, tokenizer):
    """
    Turn word indices into text using the tokenizer
    :param idx: word indices
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[ind] for ind in idx])

print('`indices_to_text` function loaded.')

`indices_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [16]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 0.03
    GRU_units = 250
    
    l1 = Input(shape = input_shape[1:])
    l2 = GRU(units = GRU_units, return_sequences = True)(l1)
    # Dense layer does the TimeDistribution automatically
    # So we don't need to wrap it in TimeDistributed
    out = Dense(units = french_vocab_size, activation = 'softmax')(l2)
    
    
    model = Model(inputs = l1, outputs = out)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
# note that the max. length of french sentences is bigger than that in english
# so it makes sense to pad the input with max_french_sequence_length
# on the other hand had max. length of english sentences been greater, it 
# would make sense to pad the french sentences with max_english_length 
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))




Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 110288 samples, validate on 27573 samples
Epoch 1/10





110288/110288 [==============================] - 45s 406us/step - loss: 1.6509 - acc: 0.5766 - val_loss: 1.2640 - val_acc: 0.6239
Epoch 2/10
110288/110288 [==============================] - 45s 408us/step - loss: 1.1313 - acc: 0.6444 - val_loss: 1.0611 - val_acc: 0.6580
Epoch 3/10
110288/110288 [==============================] - 45s 404us/step - loss: 1.0433 - acc: 0.6557 - val_loss: 1.0012 - val_acc: 0.6566
Epoch 4/10
110288/110288 [==============================] - 45s 405us/step - loss: 0.9955 - acc: 0.6641 - val_loss: 0.9576 - val_acc: 0.6747
Epoch 5/10
110

In [17]:
indices_to_text(preproc_english_sentences[0], english_tokenizer)

'new jersey is sometimes quiet during autumn , and it is snowy in april . <PAD> <PAD>'

In [18]:
indices_to_text(preproc_french_sentences[0].reshape(-1,), french_tokenizer)

"new jersey est parfois calme pendant l' automne , et il est neigeux en avril . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [19]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    embedding_dim = 70
    GRU_units = 250
    learning_rate = 0.03
    
    l1 = Input(shape = (input_shape[1],))
    l2 = Embedding(input_dim = english_vocab_size, 
                   output_dim = embedding_dim, 
                   input_length = input_shape[1])(l1)
    l3 = GRU(units = GRU_units, return_sequences = True)(l2)
    out = Dense(units = french_vocab_size, activation = 'softmax')(l3)
    
    model = Model(inputs = l1, outputs = out)
    
    model.compile(optimizer = Adam(learning_rate), 
                  loss = sparse_categorical_crossentropy, metrics = ['accuracy'])
    
    return model
tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
embd_model = embed_model(tmp_x.shape, 
                         max_french_sequence_length, 
                         english_vocab_size, french_vocab_size ) 

# TODO: Train the neural network

embd_model.fit(tmp_x, preproc_french_sentences, epochs = 5, 
               batch_size = 1024, validation_split = 0.2 )

# TODO: Print prediction(s)
# I noticed that the accuracy of the model on the validation set started dropping after 5 epochs
# so we will only train it for 5 epochs
print(logits_to_text(embd_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/5
110288/110288 [==============================] - 49s 447us/step - loss: 1.7410 - acc: 0.6116 - val_loss: 0.6957 - val_acc: 0.7934
Epoch 2/5
110288/110288 [==============================] - 49s 445us/step - loss: 0.5418 - acc: 0.8324 - val_loss: 0.4314 - val_acc: 0.8643
Epoch 3/5
110288/110288 [==============================] - 49s 444us/step - loss: 0.3971 - acc: 0.8731 - val_loss: 0.3766 - val_acc: 0.8814
Epoch 4/5
110288/110288 [==============================] - 48s 439us/step - loss: 0.3432 - acc: 0.8888 - val_loss: 0.3226 - val_acc: 0.8960
Epoch 5/5
110288/110288 [==============================] - 48s 438us/step - loss: 0.3126 - acc: 0.8983 - val_loss: 0.3085 - val_acc: 0.9009
new jersey est parfois calme en l' automne , et il est neigeux en avril . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [20]:
indices_to_text(preproc_english_sentences[0], english_tokenizer)

'new jersey is sometimes quiet during autumn , and it is snowy in april . <PAD> <PAD>'

In [21]:
indices_to_text(preproc_french_sentences[0].reshape(-1,), french_tokenizer)

"new jersey est parfois calme pendant l' automne , et il est neigeux en avril . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [22]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 0.005
    GRU_units = 250
    bi_mode = 'mul'
    hidden_units = 500
    
    l1 = Input(shape = input_shape[1:])
    l2 = Bidirectional(GRU(units = GRU_units, return_sequences = True), 
                       merge_mode = bi_mode)(l1)
    l3 = Dense(units = hidden_units, activation = 'elu')(l2)
    out = Dense(units = french_vocab_size, activation = 'softmax')(l3)
    
    model = Model(inputs = l1, outputs = out)
    
    model.compile(optimizer = Adam(learning_rate), 
                  loss = sparse_categorical_crossentropy, 
                  metrics = ['accuracy'])
    
    
    return model
tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape(-1, max_french_sequence_length, 1)
bidir_model = bd_model(tmp_x.shape, max_french_sequence_length, 
                       english_vocab_size, french_vocab_size)
# this model seems to start overfitting after 9 epochs
bidir_model.fit(tmp_x, preproc_french_sentences, epochs = 9, batch_size = 512, 
                validation_split = 0.2)

# print a prediction
print(logits_to_text(bidir_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/9
110288/110288 [==============================] - 86s 776us/step - loss: 1.3958 - acc: 0.6275 - val_loss: 1.0098 - val_acc: 0.6663
Epoch 2/9
110288/110288 [==============================] - 84s 763us/step - loss: 0.9276 - acc: 0.6851 - val_loss: 0.8654 - val_acc: 0.6949
Epoch 3/9
110288/110288 [==============================] - 81s 737us/step - loss: 0.8210 - acc: 0.7037 - val_loss: 0.7694 - val_acc: 0.7180
Epoch 4/9
110288/110288 [==============================] - 81s 733us/step - loss: 0.7435 - acc: 0.7254 - val_loss: 0.7182 - val_acc: 0.7369
Epoch 5/9
110288/110288 [==============================] - 80s 721us/step - loss: 0.6650 - acc: 0.7565 - val_loss: 0.6313 - val_acc: 0.7729
Epoch 6/9
110288/110288 [==============================] - 78s 712us/step - loss: 0.6196 - acc: 0.7758 - val_loss: 0.6069 - val_acc: 0.7738
Epoch 7/9
110288/110288 [==============================] - 78s 703us/step - loss: 0.5696 - acc: 0.7946 - val_

In [23]:
indices_to_text(preproc_english_sentences[0], english_tokenizer)

'new jersey is sometimes quiet during autumn , and it is snowy in april . <PAD> <PAD>'

In [24]:
indices_to_text(preproc_french_sentences[0].reshape(-1,), french_tokenizer)

"new jersey est parfois calme pendant l' automne , et il est neigeux en avril . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [25]:
from keras.layers import Reshape

In [26]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # the model here is based on the encoder-decoder configuration suggested 
    # in the following blog
    # https://machinelearningmastery.com/encoder-decoder-long-short-term-memory-networks/
    # The idea is that for each timestep of the decoder, we feed it 
    # the final output produced by the decoder
    # This essentially means that we feed the same input vector to the decoder at each time step
    
    # OPTIONAL: Implement
    learning_rate = 0.0005
    embedding_dim = 70
    gru_units = 500
    dec_hidden_units = 700
    
    enc_input = Input(shape = input_shape[1:])
    reshaped = Reshape(target_shape = (input_shape[1],))(enc_input)
    embedding = Embedding(input_dim = english_vocab_size, 
                          output_dim = embedding_dim, 
                          input_length = input_shape[1])(reshaped)
    enc_gru, enc_gru_state = GRU(units = gru_units,
                                 return_sequences = False,
                                 return_state = True,
                                 go_backwards = True)(embedding)
    enc_out = RepeatVector(output_sequence_length)(enc_gru)
    
    dec_gru = GRU(units = gru_units, 
                 return_sequences = True)(enc_out, initial_state = enc_gru_state)
    dec_dense = Dense(units = dec_hidden_units, activation = 'selu')(dec_gru)
    dec_out = Dense(units = french_vocab_size, activation = 'softmax' )(dec_dense)
    
    model = Model(inputs = enc_input, outputs = dec_out)
    
    model.compile(optimizer = Adam(learning_rate), 
                  loss = sparse_categorical_crossentropy, 
                  metrics = ['accuracy'])
    
    
    
    
    
    return model
tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)
# no need to pad the english sentences to have the same length as 
# the max_len of french sentences
# just reshape
#tmp_x = preproc_english_sentences[...,np.newaxis]
tmp_x = preproc_english_sentences

encdec = encdec_model(tmp_x.shape, max_french_sequence_length, 
                      english_vocab_size, french_vocab_size)

# training the model
encdec.fit(tmp_x, preproc_french_sentences, batch_size = 32, 
           epochs = 10, validation_split = 0.2)

#prediction
print(logits_to_text(encdec.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 559s 5ms/step - loss: 1.0938 - acc: 0.7020 - val_loss: 0.5949 - val_acc: 0.8172
Epoch 2/10
110288/110288 [==============================] - 574s 5ms/step - loss: 0.3881 - acc: 0.8775 - val_loss: 0.2831 - val_acc: 0.9059
Epoch 3/10
110288/110288 [==============================] - 578s 5ms/step - loss: 0.2246 - acc: 0.9258 - val_loss: 0.1944 - val_acc: 0.9366
Epoch 4/10
110288/110288 [==============================] - 576s 5ms/step - loss: 0.1722 - acc: 0.9431 - val_loss: 0.1702 - val_acc: 0.9451
Epoch 5/10
110288/110288 [==============================] - 576s 5ms/step - loss: 0.1369 - acc: 0.9553 - val_loss: 0.1322 - val_acc: 0.9575
Epoch 6/10
110288/110288 [==============================] - 576s 5ms/step - loss: 0.1141 - acc: 0.9627 - val_loss: 0.1170 - val_acc: 0.9628
Epoch 7/10
110288/110288 [==============================] - 581s 5ms/step - loss: 0.1006 - acc: 0.9677 - val_

In [27]:
indices_to_text(preproc_english_sentences[0], english_tokenizer)

'new jersey is sometimes quiet during autumn , and it is snowy in april . <PAD> <PAD>'

In [28]:
indices_to_text(preproc_french_sentences[0].reshape(-1,), french_tokenizer)

"new jersey est parfois calme pendant l' automne , et il est neigeux en avril . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [29]:
from keras.layers import Multiply

In [30]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 0.0005
    gru_units = 500
    bi_mode = 'mul'
    hidden_units = 700
    embedding_dim = 70
    
    enc_in = Input(shape = (input_shape[1],))
    embed = Embedding(input_dim = english_vocab_size, 
                   output_dim = embedding_dim,
                   input_length = input_shape[1])(enc_in)
    enc_gru = Bidirectional(GRU(units = gru_units, 
                                               return_sequences = False),
                                          merge_mode = bi_mode)(embed)
    dec_input = RepeatVector(output_sequence_length)(enc_gru)
    dec_gru = GRU(units = gru_units, 
                  return_sequences = True)(dec_input)
    hidden = Dense(units = hidden_units, activation = 'selu')(dec_gru)
    out = Dense(units = french_vocab_size, activation = 'softmax')(hidden)
    
    model = Model(inputs = enc_in, outputs = out)
    
    model.compile(optimizer = Adam(learning_rate), 
                  loss = sparse_categorical_crossentropy, 
                  metrics = ['accuracy'])
    
    
    
    return model
tests.test_model_final(model_final)


print('Final Model Loaded')
# TODO: Train the final model
tmp_x = preproc_english_sentences

fin_model = model_final(tmp_x.shape, max_french_sequence_length, 
                        english_vocab_size, french_vocab_size)

fin_model.fit(tmp_x, preproc_french_sentences, 
              batch_size = 32, epochs = 10, validation_split = 0.2)

Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 680s 6ms/step - loss: 1.3380 - acc: 0.6521 - val_loss: 0.7202 - val_acc: 0.7796
Epoch 2/10
110288/110288 [==============================] - 681s 6ms/step - loss: 0.4626 - acc: 0.8580 - val_loss: 0.3007 - val_acc: 0.9074
Epoch 3/10
110288/110288 [==============================] - 688s 6ms/step - loss: 0.2433 - acc: 0.9233 - val_loss: 0.2016 - val_acc: 0.9363
Epoch 4/10
110288/110288 [==============================] - 683s 6ms/step - loss: 0.1784 - acc: 0.9437 - val_loss: 0.1710 - val_acc: 0.9464
Epoch 5/10
110288/110288 [==============================] - 674s 6ms/step - loss: 0.1408 - acc: 0.9560 - val_loss: 0.1486 - val_acc: 0.9549
Epoch 6/10
110288/110288 [==============================] - 674s 6ms/step - loss: 0.1198 - acc: 0.9630 - val_loss: 0.1204 - val_acc: 0.9632
Epoch 7/10
110288/110288 [==============================] - 676s 6ms/step - loss: 0.1033 -

In [31]:
#prediction
print(logits_to_text(fin_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois calme pendant l' automne , et il est neigeux en avril . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Prediction (IMPLEMENTATION)

In [32]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    # I already trained it in the cell above
    model = fin_model

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Sample 1:
il a vu un vieux camion jaune . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant l' automne , et il est neigeux en avril . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne , et il est neigeux en avril . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

In [34]:
!!python -m nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 391531 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?